In [12]:
from __future__ import print_function
import lightgbm as lgb
# 导入boost集合模型中的
import sklearn
import numpy
import hyperopt
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
# hp定义的一个内置函数
# 为了能够真正看到我们需要返回的字典，让我们修改目标函数返回更多的东西，并传递一个明确的trials参数fmin,所以导入trials

import colorama
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import r2_score
N_HYPEROPT_PROBES = 200
HYPEROPT_ALGO = tpe.suggest  #  tpe.suggest OR hyperopt.rand.suggest

# ----------------------------------------------------------

colorama.init()

# ---------------------------------------------------------------------

def get_lgb_params(space):
    lgb_params = dict()
    lgb_params['boosting_type'] = space['boosting_type'] if 'boosting_type' in space else 'gbdt'
    # 默认=gbdt，类型=枚举，选项：gbdt，rf，dart，goss
    lgb_params['objective'] = 'regression'
#objective 🔗︎，默认=regression，类型=枚举，选项：regression，regression_l1，huber，fair，poisson，quantile，mape，gamma，tweedie，binary，multiclass，multiclassova，cross_entropy，cross_entropy_lambda，lambdarank，别名：objective_type，app，application
#回归应用
#regression，L2损失，别名：regression_l2，l2，mean_squared_error，mse，l2_root，root_mean_squared_error，rmse
#regression_l1，L1损失，别名：l1，mean_absolute_error，ma
    lgb_params['metric'] = 'rmse'
    # 平方损失
    lgb_params['learning_rate'] = space['learning_rate']
    # 学习率设定
    lgb_params['num_leaves'] = int(space['num_leaves'])
    # 树上最大叶子数
    lgb_params['min_data_in_leaf'] = int(space['min_data_in_leaf'])
# 叶子上最小数据
    lgb_params['min_sum_hessian_in_leaf'] = space['min_sum_hessian_in_leaf']
    # 防止过拟合
    lgb_params['max_depth'] = -1
    # 限制树的深度,<=0代表没有限制
    lgb_params['lambda_l1'] = space['lambda_l1'] if 'lambda_l1' in space else 0.0
    lgb_params['lambda_l2'] = space['lambda_l2'] if 'lambda_l2' in space else 0.0
    # 正则化
    lgb_params['max_bin'] = int(space['max_bin']) if 'max_bin' in space else 256
#  max_bin 🔗︎，default =255，type = int，约束：max_bin > 1
# 可以存储特征值的最大箱数
# 少量的垃圾桶可能会降低训练的准确性，但可能会增加总体力量（应对过度拟合
    lgb_params['feature_fraction'] = space['feature_fraction']
#     如果feature_fraction小于，LightGBM将在每次迭代（树）中随机选择部分要素1.0。例如，如果将其设置为0.8，则LightGBM将在训练每棵树之前选择80％的功能
# 可用于加快培训
# 可用于处理过度拟合
    lgb_params['bagging_fraction'] = space['bagging_fraction']
    lgb_params['bagging_freq'] = int(space['bagging_freq']) if 'bagging_freq' in space else 1
#     套袋频率
# 0表示禁用装袋；k意味着每次k迭代都执行装袋
    lgb_params['nthread'] = 4
#     LightGBM的线程数
# 0 表示OpenMP中的默认线程数
# 为了获得最佳速度，请将其设置为实际CPU内核数，而不是线程数（大多数CPU使用超线程为每个CPU内核生成2个线程）
# 如果您的数据集很小，则不要将其设置得太大（例如，对于具有10,000行的数据集，请勿使用64个线程）
# 请注意，任务管理器或任何类似的CPU监视工具可能会报告核心未得到充分利用。这是正常的
# 对于并行学习，请不要使用所有CPU内核，因为这将导致网络通信性能下降
    return lgb_params

# ---------------------------------------------------------------------
# log_writer = open( '../log/lgb-hyperopt-log.txt', 'w' )
X_train = pd.read_csv('/home/ach/下载/team-learning-master/数据竞赛（房租预测）/Filter_train_data.csv')
# print(X_train)
# print(X_train.info)
# print(X_train['0'])
Y_train = pd.read_csv('/home/ach/下载/team-learning-master/数据竞赛（房租预测）/Filter_train_data_Y.csv').pop('pre')
# new_test = test[select_columns]
obj_call_count = 0
cur_best_score = 0 # 0 or np.inf
# log_writer = open( '../log/lgb-hyperopt-log.txt', 'w' )


def objective(space):
    global obj_call_count, cur_best_score,clf

    obj_call_count += 1

    print('\nLightGBM objective call #{} cur_best_score={:7.5f}'.format(obj_call_count,cur_best_score) )

    lgb_params = get_lgb_params(space)
# 组装好
    sorted_params = sorted(space.items(), key=lambda z: z[0])
    params_str = str.join(' ', ['{}={}'.format(k, v) for k, v in sorted_params])
    print('Params: {}'.format(params_str) )
    
    kf = KFold(n_splits=3, shuffle=True, random_state=0)
    #3折交叉验证
    out_of_fold = np.zeros(len(X_train))
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train)):
        D_train = lgb.Dataset(X_train.iloc[train_idx], label=Y_train[train_idx])
        #导入数据,训练集合
        D_val = lgb.Dataset(X_train.iloc[val_idx], label=Y_train[val_idx])
        #导入
        # Train
        num_round = 10000
        clf = lgb.train(lgb_params,
                           D_train,
                           num_boost_round=num_round,
                           # metrics='mlogloss',
                           valid_sets=D_val,
                           # valid_names='val',
                           # fobj=None,
                           # feval=None,
                           # init_model=None,
                           # feature_name='auto',
                           # categorical_feature='auto',
                           early_stopping_rounds=200,
                           # evals_result=None,
                           verbose_eval=False,
                           # learning_rates=None,
                           # keep_training_booster=False,
                           # callbacks=None
                           )
        # predict
#         model = lgb.train({**self.params, **hyperparams}, train_data, self.n_est,
#                               valid_data, early_stopping_rounds=self.n_stop, verbose_eval=0)
        nb_trees = clf.best_iteration
    # 最佳索引
        val_loss = clf.best_score['valid_0']
        print('nb_trees={} val_loss={}'.format(nb_trees, val_loss))
        out_of_fold[val_idx] = clf.predict(X_train.iloc[val_idx], num_iteration=nb_trees)
        #ypred = bst.predict(data, num_iteration=bst.best_iteration)
        print(out_of_fold)
        score = r2_score(out_of_fold, Y_train)

    print('val_r2_score={}'.format(score))

#     log_writer.write('score={} Params:{} nb_trees={}\n'.format(score, params_str, nb_trees ))
#     log_writer.flush()

    if score>cur_best_score:
        cur_best_score = score
        print(colorama.Fore.GREEN + 'NEW BEST SCORE={}'.format(cur_best_score) + colorama.Fore.RESET)
    return {'loss': -score, 'status': STATUS_OK}

# --------------------------------------------------------------------------------

space ={
        'num_leaves': hp.quniform ('num_leaves', 10, 100, 1),
        'min_data_in_leaf':  hp.quniform ('min_data_in_leaf', 10, 100, 1),
        'feature_fraction': hp.uniform('feature_fraction', 0.75, 1.0),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.75, 1.0),
        'learning_rate': hp.uniform('learning_rate', 0, 0.01),
#         'learning_rate': hp.loguniform('learning_rate', -5.0, -2.3),
        'min_sum_hessian_in_leaf': hp.loguniform('min_sum_hessian_in_leaf', 0, 2.3),
        'max_bin': hp.quniform ('max_bin', 88, 200, 1),
        'bagging_freq': hp.quniform ('bagging_freq', 1, 15, 1),
        'lambda_l1': hp.uniform('lambda_l1', 0, 10 ),
        'lambda_l2': hp.uniform('lambda_l2', 0, 10 ),
       }

trials = Trials()
best = hyperopt.fmin(fn=objective,
                     space=space,
                     algo=HYPEROPT_ALGO,
                     max_evals=N_HYPEROPT_PROBES,
                     trials=trials,
                     verbose=1)

print('-'*50)
print('The best params:')
print( best )
print('\n\n')

                                                       
LightGBM objective call #1 cur_best_score=0.00000
Params: bagging_fraction=0.9444600266766812 bagging_freq=12.0 feature_fraction=0.8834815444168265 lambda_l1=7.447333786412393 lambda_l2=8.582042063499767 learning_rate=0.003707223757152437 max_bin=155.0 min_data_in_leaf=30.0 min_sum_hessian_in_leaf=3.081175187107079 num_leaves=65.0
nb_trees=8302 val_loss=OrderedDict([('rmse', 722.4208707320247)])
[   0.            0.            0.         ...    0.         2582.19121532
 3480.17782588]
nb_trees=9920 val_loss=OrderedDict([('rmse', 743.7906464980161)])
[ 5894.51485168  2419.99729086 12642.96780906 ...     0.
  2582.19121532  3480.17782588]
nb_trees=7236 val_loss=OrderedDict([('rmse', 755.0665503855082)])
[ 5894.51485168  2419.99729086 12642.96780906 ...  3339.46282831
  2582.19121532  3480.17782588]
val_r2_score=0.9028789540400711                        
NEW BEST SCORE=0.9028789540400711                      
                        

KeyboardInterrupt: 